# Main Chatbot

Handle imports and load data.

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=5977ead2d1fcc51a1b29efccb5d2f19e40a0db7ddf7449d43fe1feda12f0eda3
  Stored in directory: /tmp/pip-ephem-wheel-cache-oeuou8qc/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab

In [38]:
import numpy as np
import pandas as pd
import re
import spacy
import random
import math
import warnings
import sklearn.svm
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.neural_network
import sklearn.neighbors
import sklearn.ensemble
import sklearn.preprocessing

In [58]:
database = pd.read_csv('/content/drive/MyDrive/IRTStuff/database.csv')
def retrieve(course, identifier):
    i = database[(database['id']==course) | (database['name']==course)]
    i = i[identifier].item()
    return i

In [63]:
# print(database)
print(retrieve('01:198:105', 'credits'))

3


In [33]:
nlp = spacy.load("en_core_web_lg")


In [159]:
class Query:
    name = ""
    examples = []

    @staticmethod
    def response():
       print(tempvar)

    children = {}
    model = None
    decoder = None
    responsegenerator = {}

    def responses(self):
      return {'response': getattr(self,'response')}

    def __init__(self, name, examples, response, children = []):
        self.name = name
        self.examples = [nlp(x) for x in examples]
        self.response = response
        self.responsegenerator['response'] = classmethod(response)
        test = [[(e.text, e.label_) for e in d.ents] for d in self.examples]
        print(test)
        self.children = dict(zip(map(lambda x : x.name, children), children))
        if len(self.children) > 0:
            self.sktrainQuery()

    def addChild(self, child, suppressTrain=False):
        if child.name in self.children:
            return None

        self.children[child.name] = child

        if not suppressTrain:
            self.sktrainQuery()

    def addChildren(self, childlist):
        for childQ in childlist:
            self.addChild(childQ, suppressTrain=True)

        self.sktrainQuery()

    def removeChild(self, name, suppressTrain=False):
        self.children.discard(name)

        if not suppressTrain:
            self.sktrainQuery()

    def removeChildren(self, namelist):
        for name in namelist:
            self.removeChild(name, suppressTrain=True)

        if len(namelist) > 0:
            self.sktrainQuery()



    def trainingData(self):
        # return [[s, self.name] for s in [x.vector.reshape(1,-1) for x in self.examples]]
        return [[s, self.name] for s in self.examples]

    def queriessqrt(self):
        return round(math.sqrt(len(self.children)))

    def getModel(self):
        print('getting model')
        rfc = sklearn.ensemble.RandomForestClassifier(max_features='auto')
        # gnbc = sklearn.naive_bayes.GaussianNB()
        sgdc = sklearn.linear_model.SGDClassifier()
        ptronc = sklearn.linear_model.SGDClassifier(loss='perceptron')
        # knc = sklearn.neighbors.KNeighborsClassifier()


        # combinedclassifier = sklearn.naive_bayes.GaussianNB()
        combinedclassifier = sklearn.linear_model.SGDClassifier(penalty='elasticnet', loss='hinge')

        ensemblec = sklearn.ensemble.StackingClassifier(estimators=[('rf', rfc),
                                                                    ('sgd', sgdc),
                                                                    ('ptron', ptronc),
                                                                    # ('gnb', gnbc),
                                                                    # ('kn', knc)
                                                                    ],
                                                        final_estimator=combinedclassifier)

        params = {'cv': [None, 2],
                  'rf__n_estimators': [100, self.queriessqrt()],
                  # 'rf__criterion': ['gini', 'entropy'],
                  # 'rf__max_features': ['auto', 'log2'],
                  # 'rf__warm_start':['true'],
                  'rf__class_weight':[None, 'balanced'],
                  # 'sgd__loss': ['hinge', 'log'],
                  'sgd__penalty': ['l2', 'elasticnet'],
                  # 'kn__n_neighbors': [5, self.queriessqrt()],
                  'final_estimator__warm_start': [False]
                  }
        gc = sklearn.model_selection.GridSearchCV(ensemblec, params)

        return sgdc


    def sktrain(self, datalist, le=None, vc=None):
        # print(datalist)
        train = datalist
        target = train[train.columns[len(train.columns)-1]]
        train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
        if le is None:
            le = sklearn.preprocessing.LabelEncoder()
            le.fit(target)
        target = le.transform(target)
        # print(train.values.tolist())
        # print(train)
        # train = pd.DataFrame(list(train.apply(lambda x : x.vector.reshape(1,-1), axis=1, result_type='reduce')))
        train = np.stack([d[0][0] for d in train.values.tolist()])
        # print("train")
        # print(train)
        # print("target")
        # print(target)
        # print()
        if vc is None:
            vc = sklearn.linear_model.SGDClassifier()
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                vc = self.getModel()
        vc.fit(train, target)


        # vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(input='content', use_idf=True, ngram_range=(1,2))

        self.model = vc
        self.decoder = le
        return vc, le



    def sktrainQuery(self):
        queries = self.children.values()
        classifier = None
        le = None
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            data = pd.concat([pd.DataFrame(q.trainingData(), columns=['data', 'target']) for q in queries])
            data.reset_index(drop=True, inplace=True)
            data['data'] = pd.Series([x.vector.reshape(1,-1) for x in data['data'].tolist()])
            # print(data)
            classifier, le = self.sktrain(data)
            # print('error not there')
        return classifier, le

    def sktestQuery(self, cl=None, le=None):
        queries = self.children.values()
        data = pd.concat([pd.DataFrame(q.trainingData(), columns=['data', 'target']) for q in queries])
        # datalist = pd.DataFrame([q.trainingData() for q in list(queries.values())], columns = ['data', 'target'])
        data.reset_index(drop=True, inplace=True)
        datalist = data.copy()
        datalist['data'] = pd.Series([x.vector.reshape(1,-1) for x in data['data'].tolist()])

        traindatalist = pd.DataFrame(columns=['data', 'target'])
        testdatalist = pd.DataFrame(columns=['data', 'target'])

        idxs = random.sample(range(len(datalist)), min(round(0.6*len(datalist)), len(datalist)))
        traindatalist = datalist.iloc[idxs]
        testdatalist = datalist.drop(labels=idxs, axis=0)

    # while not all(traindatalist['target'].tolist().count(elem) > 2 for elem in datalist['target'].tolist()):
    #     traindatalist = datalist.sample(frac=0.6, replace=False)

    # while not all(testdatalist['target'].tolist().count(elem) > 2 for elem in datalist['target'].tolist()):
    #     testdatalist = datalist.sample(frac=0.4, replace=False)


    # print(traindatalist)
    # print(testdatalist)


    # temp = []
    # rowsSelected = []

    # print(datalist)
    # for i in range(len(datalist)):
    #     print(i)
    #     print(datalist['target'][i])
    #     if datalist['target'][i] not in temp:
    #         temp.append(datalist['target'][i])
    #         rowsSelected.append(i)
    #         print('added')
    #     print('not added. useless.')

    # # rowsNotSelected = [i for i in list(range(len(datalist))) if i not in rowsSelected]
    # temp = np.random.choice(temp, round(len(datalist)*0.5), replace=False)

    # rowsSelected = rowsSelected.extend(temp)
    # # rowsNotSelected = [i for i in list(range(len(datalist))) if i not in rowsSelected]

    # traindatalist = datalist.iloc[rowsSelected]
    # print(len(traindatalist))
    # testdatalist = datalist.drop(index=rowsSelected)


    # print(len(testdatalist))
    # print(traindatalist)
    # print(testdatalist)
    # print(datalist)
    # print(rowsSelected)


    # print("train data lsit")
    # print(traindatalist)
    # print(traindatalist['data'].to_numpy())

    # print("actual dataframe generated")
    # print(pd.DataFrame(traindatalist['data'].to_numpy()))

    # print('train columns')
    # print(traindatalist.columns)


        print('training...')
        cl, le = sktrain(traindatalist, vc=cl, le=le)

        testdatalist['target'] = le.transform(testdatalist['target'])


        score = cl.score(np.stack([d[0] for d in testdatalist['data'].values.tolist()]), testdatalist['target'])

        print("sklearn classifier scored " + str(score))
        # print("data was")
        # print(data)
        # print()
        # print('test indices are ')
        # print(testdatalist.index)
        # print(data['data'][1])
        data.columns = ['dataname', 'target']

        printtestdatalist = testdatalist.copy()
        printtestdatalist['target'] = le.inverse_transform(testdatalist['target'])
        printtestdatalist = printtestdatalist.join(data.drop('target', axis=1))
        print(printtestdatalist)
        print()

        return score, traindatalist, testdatalist, datalist



In [160]:
queries = {'greeting': Query('greeting',
                             ["hello, how are you?",
                              "greetings!",
                              "salutations!",
                              "nice to meet you!",
                              "good morning!",
                              "hello.",
                              "hi.",
                              "hi!"],
                             lambda: print("hello to you too!")),
           'farewell': Query('farewell',
                             ["goodbye.",
                              "goodbye!",
                              "see you later!",
                              "see you later, then!",
                              "farewell!"],
                             lambda: print("goodbye to you too!")),
           'whatclass': Query('whatclass',
                              ['What course should I take?',
                               'What course does it make sense for me to take?',
                               'What course should I add?',
                               'What course deals with Z?',
                               'What course can teach me about Z?',
                               'What class do you suggest?'],
                              lambda: print("whatclass")),
           'shouldtake': Query('shouldtake',
                               ['Does it make sense for me to take Discrete?',
                                'Is Discrete applicable for the Y minor?',
                                'Should I add Discrete?',
                                'Should I take Discrete?',
                                'Do I have to take Discrete?',
                                'Do I need to take Discrete?',
                                'Is Discrete required for the Y minor?',
                                'Would you recommend Discrete?',
                                'What do you think about Discrete?',
                                'What about Discrete?',
                                'How is Discrete?'],
                               lambda: print("shouldtake")),
           'shoulddrop': Query('shoulddrop',
                               ['Should I drop Discrete?',
                                'Should I drop Discrete course?',
                                'I don\'t think I should take Discrete',
                                'Can I avoid taking Discrete?',
                                'Am I able to drop Discrete?',
                                'Will I have to take Discrete or can I drop it?',
                                'Will I have to take Discrete or can I skip it?',
                                'What course can I switch for Discrete?',
                                'I don\'t think I should take Discrete. What could I take instead?',
                                'And without Discrete?',
                                'What course can I swap for Discrete?'],
                               lambda: print("shoulddrop")),
           'cantake': Query('cantake',
                            ['Can I take Discrete?',
                             'Do I have the prerequisites to take Discrete?',
                             'Can I register for Discrete?'],
                            lambda: print("cantake")),
           'info': Query('info',
                         ['Tell me about 01:198:105'],
                         lambda: print(retrieve(tempvar[14:], 'credits')))
           }


childrenmap = {'greeting': ['farewell', 'shouldtake', 'shoulddrop', 'cantake', 'whatclass'],
               'shoulddrop': ['farewell', 'shouldtake', 'shoulddrop', 'cantake', 'whatclass'],
               'shouldtake': ['farewell', 'shouldtake', 'shoulddrop', 'cantake', 'whatclass'],
               'cantake': ['farewell', 'shouldtake', 'cantake', 'whatclass']}

for k in childrenmap:
    queries[k].addChildren(queries[a] for a in childrenmap[k])

# queries['cantake'].responses = lambda self : print(self.name)
# queries['cantake'].responses()

[[], [], [], [], [('morning', 'TIME')], [], [], []]
[[], [], [], [], []]
[[], [], [], [], [], []]
[[('Discrete', 'ORG')], [], [('Discrete', 'PERSON')], [('Discrete', 'ORG')], [('Discrete', 'ORG')], [], [], [('Discrete', 'ORG')], [('Discrete', 'PERSON')], [('Discrete', 'PERSON')], [('Discrete', 'PERSON')]]
[[('Discrete', 'ORG')], [], [('Discrete', 'ORG')], [], [('Discrete', 'ORG')], [('Discrete', 'PERSON')], [('Discrete', 'PERSON')], [('Discrete', 'ORG')], [('Discrete', 'ORG')], [('Discrete', 'PERSON')], [('Discrete', 'ORG')]]
[[], [('Discrete', 'ORG')], []]
[[]]
getting model
getting model
getting model
getting model


In [157]:

def chatbot(statement):

    statement = nlp(statement)
    print(statement)

    currQ = queries['greeting']

    likelihoods = dict(map(lambda a: (a, max([statement.similarity(x) for x in currQ.children[a].examples])), currQ.children))
    print(likelihoods)
    try:
        largest = max(likelihoods, key=likelihoods.get)
    except:
        largest = None
    print(largest)
    currQ.children.get(largest, queries['farewell']).response()['response']()

    cl = currQ.model
    le = currQ.decoder

    print("sklearn")
    print(le.inverse_transform(cl.predict([statement.vector])))
    print("sklearn probabilities")
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            prob = cl.predict_proba([statement.vector]).tolist()[0]
            print(prob)
            print(le.inverse_transform(range(len(prob))))
            print(dict(zip(le.inverse_transform(range(len(prob))), prob)))
    except:
        print("probabilities not available")

    queries.get(le.inverse_transform(cl.predict([statement.vector]))[0], 'farewell').response()['response']()

tempvar = None

def run():
    global tempvar
    tempvar = ""
    leaverlist = queries['farewell'].examples
    leaverlist = leaverlist + [nlp(x) for x in ["quit.", "quit", "exit", "leave", "goodbye"]]
    while max([nlp(tempvar).similarity(leaver) for leaver in leaverlist]) < 0.85:
        tempvar = input("say something!")
        print()
        chatbot(tempvar)
        print()
        print("leaver max:")
        print(max([nlp(tempvar).similarity(leaver) for leaver in leaverlist]))
        print()
        print()

In [135]:
run()

/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

say something!quit

quit
{'farewell': 0.4383406997137953, 'shouldtake': 0.4779757250159569, 'shoulddrop': 0.5132090363117883, 'cantake': 0.4517152362767242, 'whatclass': 0.4911974812166611}
shoulddrop


NameError: ignored

In [161]:
tempvar = "Tell me about Data Structures"
temp = "Data Structures"
# retrieve(temp, 'credits')
# (lambda : print(retrieve(tempvar[14:], 'credits')))()
x = queries['info'].responses()['response']()

4


In [ ]:
df = pandas.read_csv('smallData.csv')
df.head(5)

Define retrieve function for accessing information in database

In [ ]:
def retrieve(course, identifier):
    i = df[(df['id']==course) | (df['name']==course)]
    i = i[identifier].item()
    return i

In [ ]:
# retrieve("01:198:112", "credits")

Load Spacy Model & Tokenize utterances

In [ ]:
nlp = spacy.load("en_core_web_sm")
def tokenize(utterance):
    doc = nlp(utterance)
    tagged_tokens = []
    for token in doc:
        tagged_tokens.append((token.text, token.pos_))

    return tagged_tokens

In [ ]:
tokenize("I am very happy!")

Handle Tagged & Untagged Reflections

In [ ]:
tagged_reflection_of = {
    ("you", "PPSS") : "I",
    ("you", "PPO") : "me"
}

untagged_reflection_of = {
    "am"    : "are",
    "i"     : "you",
    "i'd"   : "you would",
    "i've"  : "you have",
    "i'll"  : "you will",
    "i'm"   : "you are",
    "my"    : "your",
    "me"    : "you",
    "you've": "I have",
    "you'll": "I will",
    "you're": "I am",
    "your"  : "my",
    "yours" : "mine"
}

Translate Tokens

In [ ]:
def translate_token(wt) :
    (word, tag) = wt
    wl = word.lower()
    if (wl, tag) in tagged_reflection_of :
        return (tagged_reflection_of[wl, tag], tag)
    if wl in untagged_reflection_of :
        return (untagged_reflection_of[wl], tag)
    if tag.find("NP") < 0 :
        return (wl, tag)
    return (word, tag)

subject_tags = ["PPS",  # he, she, it | pronouns singular
                "PPSS", # you, we, they | pronouns plural
                "PN",   # everyone, someone | indefinite pronoun
                "NN",   # dog, cat | noun, common, singular or mass
                "NNS",  # dogs, cats | noun, common, plural
                "NP",   # Fred, Jane | proper noun
                "NPS"   # Republicans, Democrats | proper plural
                ]

def swap_ambiguous_verb(tagged_words, tagged_verb_form, target_subject_pronoun, replacement) :
    for i, (w, t) in enumerate(tagged_words) :
        if (w, t) == tagged_verb_form :
            j = i - 1
            # look earlier for the subject
            while j >= 0 and tagged_words[j][1] not in subject_tags :
                j = j - 1
            # if subject is the target, swap verb forms
            if j >= 0 and tagged_words[j][0].lower() == target_subject_pronoun :
                tagged_words[i] = replacement
            # didn't find a subject before the verb, so probably a question 
            if j < 0 :
                j = i + 1
                while j < len(tagged_words) and tagged_words[j][1] not in subject_tags :
                    j = j + 1
                # if subject is the target, swap verb forms
                if j < len(tagged_words) and tagged_words[j][0].lower() == target_subject_pronoun :
                    tagged_words[i] = replacement
                    
def handle_specials(tagged_words) :
    # don't keep punctuation at the end
    while tagged_words[-1][1] == 'PUNCT' :
        tagged_words.pop()
    # replace verb "be" to agree with swapped subjects
    swap_ambiguous_verb(tagged_words, ("are", "BER"), "i", ("am", "BEM"))
    swap_ambiguous_verb(tagged_words, ("am", "BEM"), "you", ("are", "BER"))
    swap_ambiguous_verb(tagged_words, ("were", "BED"), "i", ("was", "BEDZ"))
    swap_ambiguous_verb(tagged_words, ("was", "BEDZ"), "you", ("were", "BED"))

    
close_punc = ['.', ',', "''"]
def translate(this):
    '''tokens = tokenize(this)
    tagged_tokens = tagger.tag(tokens)'''
    tagged_tokens = tokenize(this)
    print(tagged_tokens)
    translation = [translate_token(tt) for tt in tagged_tokens]
    handle_specials(translation)
    if len(translation) > 0 :
        with_spaces = [translation[0][0]]
        for i in range(1, len(translation)) :
            if translation[i-1][1] != '``' and translation[i][1] not in close_punc :
                with_spaces.append(' ')
            with_spaces.append(translation[i][0])           
    return ''.join(with_spaces)

In [ ]:
translate("I am very happy!")

Eliza based responses

In [ ]:
rules = [(re.compile(x[0]), x[1]) for x in [
    ['How are you?',
         [ "I'm fine, thank you."]],
    ['Thank you!',
         ['No problem, did you have more questions?']],
    ["I had a question about (.*)",
         [  "Ask away!",
            "I am happy to help!",
            "Ok, What is your question?"]],
    ["Hello(.*)",
         [  "Hello... I'm glad you could drop by today.",
            "Hi there... how are you today?"]],
    ["I'm great", 
         ["Great! Did you want to ask me anything?",
            "Wonderful! Feel free to ask me any questions that you may have!"]],
    ["Can you answer (.*)",
         [  "Yes, I can!",
            "What is your question?"]],
    ["quit",
         [  "Thank you for talking with me.",
            "Good-bye.",
            "Have a good day!"]],
    ["(.*)",
         [  "Please tell me more.",
        "Can you elaborate on that?", "I don't understand. Can you rephrase? "]]
]]

def respond(sentence):
    # find a match among keys, last one is quaranteed to match.
    for rule, value in rules:
        match = rule.search(sentence)
        if match is not None:
            # found a match ... stuff with corresponding value
            # chosen randomly from among the available options
            resp = random.choice(value)
            # we've got a response... stuff in reflected text where indicated
            while '%' in resp:
                pos = resp.find('%')
                num = int(resp[pos+1:pos+2])
                resp = resp.replace(resp[pos:pos+2], translate(match.group(num)))
            return resp

In [ ]:
respond("I had a question about computer science?")

In [ ]:
respond("Hello, I wanted to ask about computer science.")

In [ ]:
respond("Thank you!")

Build course suggestion tree

In [ ]:
# define basic node class for course suggestion tree
class Node(object):
    
    def __init__(self, val):
        self.val = val
        self.children = []
        
    def add_child(self, obj):
        self.children.append(obj)
        
q1 = Node("suggesting courses, overview of CS or narrowed focus?")
q2 = Node("You picked overview of CS! academic discipline or applications?")
q3 = Node("You picked narrowed focus! software engineering or data science?")
q4 = Node("You picked applications! math & physical sciences or digital creation?")
q5 = Node("You picked software engineering! beginner or experienced?")
c1 = Node("You picked academic discipline! 01:198:105	Great Insights in Computer Science")
c2 = Node("You picked math & physical sciences! 01:198:107	Computing for Math and the Sciences")
c3 = Node("You picked digital creation! 01:198:110	Principles of Computer Science")
c4 = Node("You picked data science! 01:198:142	Data 101: Data Literacy")
c5 = Node("You picked beginner! 01:198:111	Introduction to Computer Science")
c6 = Node("You picked experienced! 01:198:112	Data Structures")
q1.add_child(q2)
q1.add_child(q3)
q2.add_child(c1)
q2.add_child(q4)
q4.add_child(c2)
q4.add_child(c3)
q3.add_child(c4)
q3.add_child(q5)
q5.add_child(c5)
q5.add_child(c6)

Find possible actions for robot

In [ ]:
def possible(state):
    # checking for eliza rules or task state
    plans = []
    
    # check if 0 utterances have been said
    if(len(state[utterances]) == 0):
        plans.append("Type to chat!")
    else: 
        # gets eliza response
        plans.append(respond(state[utterances][most_recent]))

    
        # check if no tasks have been initialized
        # q1Rules = ["suggest", "class", "course", "recommend"]
        q1Rules = ["recommend"]
        if(len(state[tasks]) == 0):
            for word in state[utterances][most_recent].split():
                if word in q1Rules:
                    # update_state(q1, state)
                    plans.append(q1)
                    break
        # check if tasks have been initialized -> add the next appropriate task
        elif(isinstance(state[tasks][most_recent], Node)):
            for child in state[tasks][most_recent].children:
                if state[tasks][most_recent] in child.val:
                    # update_state(child, state)
                    plans.append(child)
                    break
                    
        # information retrieval "tell me information about X"
        # infoRetrievalRules = ["information", "tell me", "describe"]
        infoRetrievalRules = ["information"]
        for rule in infoRetrievalRules:
            if rule in state[utterances][most_recent].lower():
                course = None
                identifier = None
                for id in df['id'].tolist():
                    if id.lower() in state[utterances][most_recent].lower():
                        course = id
                        break
                if course == None:
                    for name in df['name'].tolist():
                        if name.lower() in state[utterances][most_recent].lower():
                            course = name
                            break
                for i in df.columns:
                    if i.lower() in state[utterances][most_recent].lower():
                        identifier = i
                        break
                plans.append(retrieve(course, identifier))
        
    
    return plans

Find possible actions for user

In [ ]:
def userUtterance(state):
    # default plans
    plans = ["quit", "Mistake"]
    # respond to suggestion
    if len(state[tasks]) > 0 and len(state[tasks][most_recent].children) != 0: 
        tokens = re.split(", |! | or ", state[tasks][most_recent].val)
        plans = plans + tokens[-2:]
    # ask for suggestion
    if len(state[tasks]) == 0:
        # q1Rules = ["suggest", "class", "course", "recommend"]
        q1Rules = ["recommend"]
        plans.append(random.choice(q1Rules))
    
    #infoRetrievalRules = ["information", "tell me", "describe"]
    infoRetrievalRules = ["information"]
    #courses = []
    #courses.append(df["id"])
    
    # ask about class
    courses = list(df["id"]) + list(df["name"])
    #print(courses)
    utterance = random.choice(infoRetrievalRules) + " " + random.choice(df.columns) + " " + random.choice(courses)
    plans.append(utterance)
    
    return plans
    

Update States

In [ ]:
# initial_state = s0 = {"u": [], "a": [], "task": []}
initial_state = s0 = ((),(),())

In [ ]:
# define constants
utterances = 0
actions = 1
tasks = 2
most_recent = -1

In [ ]:
def update_state(task, state):
    #     state.get("task").append(task)
    #     return state
    
    u, a, t = list(state)
    t = list(t)
    t.append(task)
    t = tuple(t)
    newState = (u, a, t)
    return newState


# s1 = update_state("hello", s0)
# print(s1)

In [ ]:
def do(item, state):
#     n = state.copy()
#     n['a'] = n['a'].copy()
#     n['a'].append(item)
    
    u, a, t = list(state)
    a = list(a)
    a.append(item)
    a = tuple(a)
    newState = (u, a, t)
    return newState

# s1 = do("hello", s0)
# print(s1)

In [ ]:
def understand(item, state):
#     # cases where state needs to be updated
#     n = state.copy()
#     # check if string is not empty
#     if item:
#         n['u'] = n['u'].copy()
#         n['u'].append(item)

    u, a, t = list(state)
    u = list(u)
    if item:
        u.append(item)
    u = tuple(u)
    newState = (u, a, t)
    return newState

# s1 = understand("hello", s0)
# print(s1)

Decide random action

In [ ]:
def deliberate(plans, state):
    choice = random.choice(plans)
    if isinstance(choice, Node):
        state = update_state(choice, state)
        return choice.val
    else:
        return choice

Converse with list of utterances

In [ ]:
def converse(utterances):
#     s0 = {"u": [], "a": [], "task": []}
    s0 = ((), (), ()) # represents utterances, actions, tasks
    for utterance in utterances:
        if utterance == "quit":
            break
        s1 = understand(utterance, s0)
        plans = possible(s1)
        action = deliberate(plans, s1)
        s0 = do(action, s1)
        print(utterance)
        print(action)

In [ ]:
converse(["Hello", "I had a question about computer science", "can you suggest a course that I would like?", "overview", "academic", "quit"])

Converse with user

In [ ]:
utterance = None
# s0 = {"u": [], "a": [], "task": []}
s0 = ((), (), ()) # represents utterances, actions, tasks
while(utterance != "quit"):
    s1 = understand(utterance, s0)
    #print(s1)
    plans = possible(s1)
    action = deliberate(plans, s1)
    s0 = do(action, s1)
    #print(s0)
    utterance = input(action)

Simulate conversation between user and chatbot

In [ ]:
def simulate(state0):
    utterance = None
    while(utterance != "quit"):
        state1 = understand(utterance, state0)
        #print(state1)
        plans = possible(state1)
        action = deliberate(plans, state1)
        state0 = do(action, state1)
        #print(state0)
        print("Action: " + str(action))
        utterance = random.choice(userUtterance(state0))
        print("Uterrance: " + utterance)

In [ ]:
# s0 = {"u": [], "a": [], "task": []}
s0 = ((), (), ()) # represents utterances, actions, tasks
simulate(s0)

# Machine Learning

Define reward function

In [ ]:
leafNodes = [c1, c2, c3, c4, c5, c6]
def getReward(oldState, action, newState):
    reward = -1
    # action is in dataframe
    if (action in df.values):
        reward += 5
    else:
        # action is a leaf node
        for node in leafNodes:
            if str(action) in node.val:
                reward += 5
            
    return reward

In [ ]:
# getReward(4), getReward("Data Structures")

Determine if state is terminated

In [ ]:
def is_terminated(state):
    if len(state[utterances]) == 0:
        return False
    elif state[utterances][most_recent] == "quit":
        return True
    return False

Define Q-actions

In [ ]:
def q_action(state, actions, epsilon):
    if random.random() < epsilon:
        max_q = None
        for action in action:
            #check to see if state action pair is in q_values dictionary, if not insert it
            if (state, action) not in q_values:
                q_values.setdefault((state, action), 0)

            if q_values.get(state,action) > q_values.get(max_q):
                max_q = (state,action)
                    
        return max_q[1]
                
    else:
        return random.choice(actions)

Training

In [ ]:
# dicitionary to hold Q-values for each state & action pair(key)
q_values = {}
# {(state, action) : q_value}

# define training params
epsilon = 0.9
discount_factor = 0.9
learning_rate = 0.9

default = 4

for episode in range(100):
    # begin conversation and iterate till it finishes
    # state0 = {"u": [], "a": [], "task": []}
    state0 = ((), (), ())
    utterance = None
    
    while(True):
        # determine chatbot action
        plans = possible(state0)
        action = deliberate(plans, state0)
        userTurnState = do(action, state0)
        
        # determine user utterance
        utterance = random.choice(userUtterance(userTurnState))
        # print(utterance)
        nextState = understand(utterance, userTurnState)
        
        # handle rewards (reward for action that transitions to nextState)
        reward = getReward(state0, action, nextState)
        old_q_value = q_values.get((state0, action), default)
        
        
        # if newState is not in dictionary, add it
        found = False
        for k in q_values:
            if(k[0] == nextState):
                found = True
                break
        if not found:
            q_values[(nextState, None)] = default
        
        
        # get q_values associated with next state
        vals = []
        for key, val in q_values.items():
            if(key == nextState):
                vals.append(val)
        if(len(vals) == 0): vals.append(default)
        td = reward + (discount_factor * max(vals) - old_q_value)
        newqq = old_q_value + learning_rate * (td)
                      
        q_values.update({(state0, action):newqq})
                      
        if utterance == "quit":
            break
            
        state0 = nextState
        # print(state0)

print("Training complete!")
# print q-value dictionary
for key, value in sorted(q_values.items(), key=lambda kv: kv[1], reverse=True):
        print(key,' : ',value)

# for key, value in q_values.items():
#     print(key,' : ',value)